In [ ]:
# debug file

import random
import re

In [ ]:
#with open("BankProblem.txt") as file_pi
    #data = file_pi.read()

population = {}
dict = {}
p = 10
tol_value =  285 
gernation = 10000 
f = open("BankProblem.txt", 'r')
data = f.readlines()

In [ ]:
i = 0
while i < len(data):
    if (data[i].find("bag") == 1):
        dict[int(re.findall("\d+", data[i])[0], 10) - 1] = [float(re.findall(r"[-+]?\d*\.\d+|\d+", data[i+1])[0]), float(re.findall(r"[-+]?\d*\.\d+|\d+", data[i+2])[0])]
    i += 1  
    ## convert txt to dict with float

In [ ]:
for times in range(p): ## inital
    temp, sum = [], 0 ## sum为当前背包重量 temp为当前队列
    while (True):
        now_key = random.randint(0, 99)
        if (now_key not in temp):
            temp.append(now_key)
            sum += dict[now_key][0]
            if (sum > tol_value):
                temp.pop()
                sum -= dict[now_key][0]
                break
        # print(f'总和为{sum} 选取{now_key}bag 增加{dict[now_key][0]}')
    population[times] = temp
print(population)

In [ ]:
def mutation(kid, times):
    for _ in range(times):
        while(True):
            mutation_index = random.randint(0, 99)
            if (mutation_index not in kid):
                kid[random.randint(0, len(kid)) - 1] = mutation_index
                break
    return kid

In [ ]:
def crossover(parent_l, parent_r):
    # single point crossover
    # e = p1[0 : index] + p2[index : len(p2)]
    # f = p2[0 : index] + p1[index : len(p1)]

    index = random.randint(1, len(parent_l) - 1 if len(parent_l) < len(parent_r) else len(parent_r) - 1)
    # res = parent_l[0 : index] if random.randint(0, 1) else parent_l[index : len(parent_l)] + parent_r[0 : index] if random.randint(0, 1) else parent_r[index : len(parent_r)]
    res = parent_l[0 : index] + parent_r[index : len(parent_r)]
    return res

In [ ]:
def fitness(array_num):
    sum_fit, sum_weight = 0, 0
    sum_value = 0
    # print(array_num)
    for i in array_num:
        sum_fit += dict[i][1] / dict[i][0]
        sum_weight += dict[i][0]
        sum_value += dict[i][1]
        # sum_fit = sum_value
    # sum_ = [sum_ += dict[j][0] for i in array for j in population[i]]
    if sum_weight < 285 :
        # print(sum_fit, (tol_weight - sum_weight) / tol_weight, "fitness", sum_fit * (tol_weight - sum_weight) / tol_weight,)
        return sum_fit * sum_value
    else:
        print(f'{sum_weight} too heavy, jump it')
        return -1

In [ ]:
import sys
sys.setrecursionlimit(11000)

In [ ]:
round = 0
lowest_index = 0
lowest_fitness = fitness(population[lowest_index])
for i in population:
    if (fitness(population[i]) < lowest_fitness):
        lowest_fitness = fitness(population[i])
        lowest_index = i
        
print(lowest_fitness, lowest_index)

In [ ]:
def main():
    # binary tournament
    global round, lowest_index, lowest_fitness
    # a = [random.randint(0, p - 1) for _ in range(2)]
    # parents = [random.randint(0, p - 1) for _ in range(2)]
    while (True):
        i = random.randint(0, len(population) - 1)
        j = random.randint(0, len(population) - 1)
        if i != j:
            break
        
    p1, p2 = population[i], population[j]
    
    print(f'round {round + 1}')

    e = crossover(p1, p2)
    f = crossover(p1, p2)

    e = mutation(e, 1)
    f = mutation(f, 1)
    
    if (fitness(e) > fitness(f)):
        child = e
    else:
        child = f

    child_old = child
    child = []
    for i in child_old:
        if i not in child:
            child.append(i)


    if (fitness(child) > fitness(population[lowest_index])):
        population[lowest_index] = child
        lowest_fitness = fitness(child)
        print(f'pop {lowest_index} will be changed')
        
        for i in population:
            if (fitness(population[i]) < fitness(population[lowest_index])):
                lowest_index = i
                lowest_fitness = fitness(population[i])
            
    round += 1
    if (round < gernation): main()

In [ ]:
main()

In [ ]:
print(population)

index, sum_value = 0, 0

for i in population:
    sum = 0
    sum_weight = 0
    for j in population[i]:
        sum += dict[j][1]
        sum_weight += dict[j][0]
    
    print(i, "value is", sum, "weight is", sum_weight, "fitness is",  fitness(population[i]))
    if (sum > sum_value):
        index = i
        sum_value = sum

print("max", index, sum_value)